In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
class mybrowser():
    def __init__(self):
        # options物件主要用途為取消網頁中的彈出視窗，避免妨礙網路爬蟲的執行。
        options = Options()
        options.add_argument("--disable-notifications")

        # 第9行就是建立webdriver物件，傳入剛剛所下載的「瀏覽器驅動程式路徑」及「瀏覽器設定(chrome_options)」，
        # 其中的「瀏覽器驅動程式路徑」一定要傳入，而「瀏覽器設定(chrome_options)」則可視情況傳入，為選擇性的。
        path = 'C:/Users/Stephan/'
        self.chrome = webdriver.Chrome(path + 'chromedriver.exe', options=options)
        self.chrome.get("https://fhy.wra.gov.tw/ReservoirPage_2011/StorageCapacity.aspx")
    
    def select_reservoir(self, month, day):
        # 抓取下拉選單元件(水庫數量)
        reservoir = self.chrome.find_element_by_id("ctl00_cphMain_cboSearch")        
        select = Select(reservoir)
        # 取第二個選項
        select.select_by_index(1)
        time.sleep(2)
        
        # 抓取下拉選單元件 (月份)
        the_element = self.chrome.find_element_by_id("ctl00_cphMain_ucDate_cboMonth")
        select_month = Select(the_element)
        # 取第選項value 
        select_month.select_by_visible_text(month)
        time.sleep(2)
        
        # 抓取下拉選單元件(日期)
        the_element = self.chrome.find_element_by_id("ctl00_cphMain_ucDate_cboDay")
        select_month = Select(the_element)
        # 取第選項value 
        select_month.select_by_visible_text(day)
        time.sleep(2)
        
    def run_search(self):
        search = self.chrome.find_element_by_id("ctl00_cphMain_btnQuery")
        search.click()
        # loading 時間
        time.sleep(3)
        soup = BeautifulSoup(self.chrome.page_source, 'html.parser')
        self.chrome.close()
        return soup        

In [3]:
def one_day(month, day, mydata = None):
    browser = mybrowser()
    browser.select_reservoir(month, day)
    soup = browser.run_search()

    tbls=soup.find('table')
    trs=tbls.find_all('tr')

    # Create empty data frame with reservoir as column
    reservoir_name = []
    for i in range (2, len(trs)-1):
        reservoir_one =trs[i].find('td').get_text()
        reservoir_name.append(reservoir_one)
    
    # Get the original column
    column=trs[1].find_all('th')
    column_list = [td.get_text() for td in column]
    
    # for first time
    if mydata == None:        
        # create dictionary with the original column as key
        mydata = {} 
        df = pd.DataFrame(columns = reservoir_name)
        value = [df for i in range(len(column_list))]
        mydata = dict(zip(column_list,value))        
    
    # turn the html text into numpy array
    data = []
    for i in range (2, len(trs)-1):
        tds=trs[i].find_all('td')
        ls = []
        for j in range(1, len(tds)):
            ls.append(tds[j].get_text())
        data.append(ls)
    df = np.array(data)
    df = df.T
    
    for i, one_att in enumerate(df):
        old = mydata.get(column_list[i]) # get the dataframe of ith column
        line = datetime(2021, int(month), int(day)) # date for index
        new_row = pd.DataFrame([one_att], columns = reservoir_name, index=[line])
        new = pd.concat([old, pd.DataFrame(new_row)], ignore_index=False)
        mydata[column_list[i]] = new
    return mydata

In [4]:
mydata = None
for month in range (1, 5):
    for day in range (1, 32):
        if month == 2 and day > 28:
            continue
        elif month == 4 and day > 14:
            continue
        month_st, day_st = str(month), str(day)
        mydata = one_day(month_st, day_st, mydata)        

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="ctl00_cphMain_cboSearch"]"}
  (Session info: chrome=89.0.4389.128)


In [27]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('reservoir_info.xlsx', engine='xlsxwriter')
# Write each dataframe to a different worksheet.
mydata['出水量(萬立方公尺)'].to_excel(writer, sheet_name='outwater', encoding = 'utf-8-sig')
mydata['進水量(萬立方公尺)'].to_excel(writer, sheet_name='inwater', encoding = 'utf-8-sig')
mydata['有效蓄水量(萬立方公尺)'].to_excel(writer, sheet_name='current_level', encoding = 'utf-8-sig')
mydata['有效容量(萬立方公尺)'].to_excel(writer, sheet_name='capacity', encoding = 'utf-8-sig')
mydata['蓄水量百分比(%)'].to_excel(writer, sheet_name='percentage', encoding = 'utf-8-sig')
mydata['集水區降雨量(毫米)'].to_excel(writer, sheet_name='rain', encoding = 'utf-8-sig')
writer.save()
'''
mydata['出水量(萬立方公尺)'].to_csv('outwater.csv', encoding = 'utf-8-sig')
mydata['進水量(萬立方公尺)'].to_csv('inwater.csv', encoding = 'utf-8-sig')
mydata['有效蓄水量(萬立方公尺)'].to_csv('currentwater.csv', encoding = 'utf-8-sig')
mydata['有效容量(萬立方公尺)'].to_csv('capacity.csv', encoding = 'utf-8-sig')
mydata['蓄水量百分比(%)'].to_csv('percentage.csv', encoding = 'utf-8-sig')
mydata['集水區降雨量(毫米)'].to_csv('raining.csv', encoding = 'utf-8-sig')
'''

"\nmydata['出水量(萬立方公尺)'].to_csv('outwater.csv', encoding = 'utf-8-sig')\nmydata['進水量(萬立方公尺)'].to_csv('inwater.csv', encoding = 'utf-8-sig')\nmydata['有效蓄水量(萬立方公尺)'].to_csv('currentwater.csv', encoding = 'utf-8-sig')\nmydata['有效容量(萬立方公尺)'].to_csv('capacity.csv', encoding = 'utf-8-sig')\nmydata['蓄水量百分比(%)'].to_csv('percentage.csv', encoding = 'utf-8-sig')\nmydata['集水區降雨量(毫米)'].to_csv('raining.csv', encoding = 'utf-8-sig')\n"